# **Microsoft Defender Threat Intelligence**

## Jupyter NoteBook using the MDTI and OpenSquat to check for new domains that are potential typosquating and then to run a reputation check on those domains.

#### Author: 
Dennis Mercer  
Sr Program Manager  
Microsoft CxE MDTI  

##### Microsoft Defender Threat Intelligence (MDTI) is a complete threat intelligence platform that enables security professionals to ingest, analyze and act upon trillions of signals collected from across the internet and processed by security experts and AI. MDTI allows users to uncover and understand the global threat landscape as it relates to their organization, including context around vulnerabilities, threat actors, and the threat infrastructure that might be used to attack them.

##### Microsoft processes over 65-trillion signals every day, which is used to populate the database which MDTI pulls datasets of threat intelligence. The key to operationalizing threat intelligence is to make it not just applicable, but actionable.   This notebook seeks to make threat intelligence actionable for SOC Analyst, Threat Hunters and Cyber Threat Intelligence Researchers.

##### With this Jupyter Notebook, you can call the Reputation Endpoint and return the reputation classification and score.  Any domains that are returned by OpenSquat are then checked against the reputation endpoint.  You could use other typosquating service, you will have to modify the code to enable that service provider.  Additionally, you can use additional APIs to create additional datasets for your notebook.



________________________________________________________________________________________________________________________________________________________
### Import Statements for any potential calls within the cells below or any modifications that could be made to the source of the Typosquat Services

In [ ]:
import asyncio
from azure.identity.aio import ClientSecretCredential
from kiota_authentication_azure.azure_identity_authentication_provider import AzureIdentityAuthenticationProvider
from msgraph import GraphRequestAdapter
from msgraph import GraphServiceClient
from IPython.display import display
import ipywidgets as widgets
from azure.identity import ClientSecretCredential
import requests
import pandas as pd
import datetime
from datetime import timedelta
import matplotlib.pyplot as plt
import seaborn as sns

### You will need to add in your own OpenSquat API Key as well as your Azure Tenant ID, Client ID, and Client Secret.  For futher information on the method for protecting your Client Secret, please review the readme file for this Notebook.

In [ ]:
# The modified OpenSquat API script
def fetch_domains(keyword):
    url = f"https://opensquat1.p.rapidapi.com/{keyword}"

    headers = {
        "X-RapidAPI-Key": "Enter your OpenSquart API Key",
        "X-RapidAPI-Host": "opensquat1.p.rapidapi.com"
    }

    try:
        params = {
            "q": keyword
        }
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()  # Raise an exception for 4xx and 5xx status codes
        data = response.json()

        # Extracting the domains from the response
        domains = data.get("domains")
        return domains
    except requests.exceptions.RequestException as error:
        print(error)

# Get the client secret from a local file
credential = ClientSecretCredential(
    tenant_id='Enter your Azure Tenant ID',  
    client_id='Enter your App Client ID',
    client_secret = 'Enter your Client Secret'  # You should not leave the client secret exposed, delete the secret one you have used it or employ Key Vault
)

scopes = ['https://graph.microsoft.com/.default']  # Scopes or permissions required for API access

# The get_reputation function
async def get_reputation(hostname: str):
    auth_provider = AzureIdentityAuthenticationProvider(credential, scopes=scopes)
    request_adapter = GraphRequestAdapter(auth_provider)

    client = GraphServiceClient(request_adapter)

    reputation = await client.security.threat_intelligence.hosts.by_host_id(hostname).reputation.get()
    if reputation:
        classification = str(reputation.classification).split('.')[-1]  # Get only the classification part
        print(f"The domain {hostname} is {classification} with a score of {reputation.score}")

try:
    loop = asyncio.get_running_loop()
except RuntimeError:  # 'RuntimeError: There is no current event loop...'
    loop = None

# Use the fetch_domains function to get the list of domains
keyword = input("Enter the keyword to search: ")
domains = fetch_domains(keyword)

print("Domains to check:", domains)  # Print the array of domains

# Use the get_reputation function to get the reputation of the domains
for domain in domains:
    if loop and loop.is_running():
        tsk = loop.create_task(get_reputation(domain))
    else:
        asyncio.run(get_reputation(domain))
